In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


position encoding + [convolution-layer × # + self-attention-layer + feed-forward-layer]

> We use depthwise separable convolutions (Chollet, 2016) (Kaiser et al., 2017) rather than traditional ones, as we observe that it is memory efficient and has better generalization.

これはkerasが持ってくれている - [SeparableConv2D](https://keras.io/layers/convolutional/#separableconv2d)

> For the self-attention-layer, we adopt the multi-head attention mechanism

これは自前実装が必要そう

勉強だから自分で実装するけど、答えは[ここ](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py#L2557)

> A positional encoding is added to the input at the beginning of each encoder layer consisting of sin and cos functions at varying wavelengths, as defined in (Vaswani et al., 2017a). 

これも自前かな？元論文にあたる

tensor2tensorに[実装](https://github.com/tensorflow/tensor2tensor/blob/ed9e3bdfd0292d4b6e5b1a1bf272146c8e2f5e9f/tensor2tensor/layers/common_attention.py#L386)がある。これ使わせてもらいたいなぁ

tensor2tensor自体をpip installしてimportできちゃった、超横着

(これやりだすとmulti-head attentionだってimportできちゃうんじゃないのか？勉強したいから自分で書くつもりだけど…)

> Each of these basic operations (conv/self-attention/ffn) is placed inside a residual block

これも自前かな？

ドロップアウト系は後で見直しが必要だな

layernormとかresidual networkとかはtransformerでもやっている

In [2]:
import tensor2tensor.layers.common_attention

In [3]:
tensor2tensor.layers.common_attention.get_timing_signal_1d?

In [4]:
# Scaled Dot-Product Attention

d_k = 20
d_v = 24
n = 5
batch_size = 2

def dot_product_attention(Q, K, V):
    d_k = tf.cast(K.shape[-1], tf.float32)
    return tf.matmul(tf.nn.softmax(
        tf.matmul(Q, K, transpose_b=True) / tf.square(d_k)), V)

In [5]:
np.random.seed(1234)

Q = np.random.randn(batch_size, n, d_k).astype(np.float32)
K = np.random.randn(batch_size, n, d_k).astype(np.float32)
V = np.random.randn(batch_size, n, d_v).astype(np.float32)

dot_product_attention(Q, K, V)

<tf.Tensor: id=22, shape=(2, 5, 24), dtype=float32, numpy=
array([[[-0.2755355 ,  0.02543714, -0.6558655 , -0.39848557,
          0.08035114, -0.21854311, -0.06740077,  0.31412017,
          0.00938186,  0.12228555, -0.8029323 ,  0.4190522 ,
         -0.01590724, -0.4729288 , -0.2317182 , -0.1456897 ,
          0.25555655,  0.16826384, -0.43338117,  0.03029106,
         -0.3693037 , -0.5143404 ,  0.60283834, -0.08868167],
        [-0.2725145 ,  0.01626787, -0.6460343 , -0.41061652,
          0.077563  , -0.2134846 , -0.05939922,  0.32173946,
          0.023509  ,  0.12638085, -0.8015075 ,  0.43884793,
         -0.01859665, -0.46918342, -0.22741178, -0.15097655,
          0.2623397 ,  0.17699657, -0.45197996,  0.02225801,
         -0.36940283, -0.50536317,  0.5990656 , -0.0851727 ],
        [-0.277115  ,  0.01595478, -0.6528961 , -0.40403602,
          0.07815958, -0.21463348, -0.06500801,  0.31738865,
          0.01783026,  0.12823199, -0.80387074,  0.4270888 ,
         -0.01027798, -0

In [6]:
n_heads = 3

np.random.seed(1234)

Q = np.random.randn(batch_size, n_heads, n, d_k).astype(np.float32)
K = np.random.randn(batch_size, n_heads, n, d_k).astype(np.float32)
V = np.random.randn(batch_size, n_heads, n, d_v).astype(np.float32)

dot_product_attention(Q, K, V)

<tf.Tensor: id=43, shape=(2, 3, 5, 24), dtype=float32, numpy=
array([[[[-4.64355499e-01,  2.69382715e-01, -4.61483561e-02,
           1.88717410e-01,  1.37169406e-01, -2.60713875e-01,
           8.99664998e-01,  4.32149261e-01,  1.04205871e+00,
          -9.48710382e-01,  3.28569591e-01,  1.01595843e+00,
           1.54692337e-01,  3.78855169e-01,  7.48895109e-01,
           9.90934968e-02, -7.83676803e-01,  1.07098937e+00,
           2.24177644e-01, -2.60871798e-01, -2.38526314e-01,
          -6.25384003e-02,  4.02206965e-02,  2.06599742e-01],
         [-4.51582968e-01,  2.87480295e-01, -8.14244375e-02,
           1.72389135e-01,  1.54843479e-01, -2.63549954e-01,
           9.22879934e-01,  4.21254009e-01,  1.03930056e+00,
          -9.52462614e-01,  3.44345272e-01,  1.03775787e+00,
           1.55778974e-01,  4.01335001e-01,  7.56202757e-01,
           9.12728831e-02, -8.02596569e-01,  1.07786441e+00,
           2.31840387e-01, -2.67707944e-01, -2.39055395e-01,
          -4.78027388e

In [8]:
np.random.seed(1234)

d_model = 20

Q = np.random.randn(batch_size, n, d_model)
W_Q = np.random.randn(d_model, d_k)

print(Q.shape, W_Q.shape)

(2, 5, 20) (20, 20)


In [9]:
[Q.shape[-1]]

[20]

In [10]:
np.dot(Q[0], W_Q)

array([[-6.02186368e+00,  4.71036170e+00,  4.78394407e-01,
        -1.95374941e+00, -3.77729152e+00, -9.03798581e-01,
         9.29919941e+00,  4.07358160e+00,  6.91877445e+00,
        -1.03789112e+00, -5.33030335e+00,  2.94333103e-03,
         8.36836844e-01, -2.43775123e+00,  1.69049897e+00,
        -9.72307875e-01, -1.01871267e+00,  5.45488202e+00,
        -2.25399293e+00, -1.11217924e+01],
       [-8.12207869e-01, -4.47643595e+00, -2.39715340e+00,
         2.19649059e+00, -7.79263434e-01,  8.36597112e-01,
         9.88349936e-01, -1.51310167e-01, -6.97845228e+00,
         3.78534711e-01,  3.59889430e+00, -3.08007472e+00,
         2.68039262e+00, -1.01589415e+00, -1.61473832e+00,
         3.14471846e+00, -5.26927758e+00, -8.46149372e+00,
        -2.40889099e+00,  4.95341156e+00],
       [-2.29957305e+00, -2.13494800e+00,  1.07428170e+00,
        -6.38390754e+00,  3.69538366e+00,  4.92677788e-01,
        -3.51141490e+00, -4.40223549e+00,  4.43084002e+00,
         3.17779349e+00,  2.9

In [11]:
assert all([all(np.isclose(x, y)) for x, y in zip(
    tf.tensordot(Q, W_Q, [[len(Q.shape) - 1], [0]])[0],
    np.dot(Q[0], W_Q))])

In [12]:
assert all([all(np.isclose(x, y)) for x, y in zip(
    tf.tensordot(Q, W_Q, [[len(Q.shape) - 1], [0]])[1],
    np.dot(Q[1], W_Q))])

In [61]:
def split_heads(x, num_heads):
    """xの最後の次元をnum_headsに分ける

    引数:
      x: (batch_size, N, num_heads * dim)
    戻り値: (batch_size, num_heads, N, dim)
    """
    shape = x.shape.as_list()
    # (batch_size, N, num_heads, dim)
    splitted = tf.reshape(
        x, shape[:-1] + [num_heads, shape[-1] // num_heads])
    # (batch_size, num_heads, N, dim)
    return tf.transpose(splitted, [0, 2, 1, 3])

def combine_heads(x):
    """xの最後の次元をnum_heads * d_vに戻す
    
    引数:
      x: (batch_size, num_heads, N, d_v)
    戻り値: (batch_size, N, num_heads * d_v)
    """
    x = tf.transpose(x, [0, 2, 1, 3])
    shape = x.shape.as_list()
    return tf.reshape(x, shape[:-2] + [shape[-2] * shape[-1]])

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, d_k, d_v, **kwargs):
        self._num_heads = num_heads
        self._d_k = d_k
        self._d_v = d_v
        super(MultiHeadAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        Q_shape, _, _ = input_shape
        d_model = Q_shape[-1]
        
        self._W_Q = self.add_weight(
            'W_Q',
            [d_model, self._num_heads * self._d_k],
            initializer='glorot_uniform')
        self._W_K = self.add_weight(
            'W_K',
            [d_model, self._num_heads * self._d_k],
            initializer='glorot_uniform')
        self._W_V = self.add_weight(
            'W_V',
            [d_model, self._num_heads * self._d_v],
            initializer='glorot_uniform')
        self._W_O = self.add_weight(
            'W_O',
            [self._num_heads * self._d_v, d_model],
            initializer='glorot_uniform')
        
        return super(MultiHeadAttention, self).build(input_shape)
    
    def call(self, x):
        # (batch_size, N, d_model)
        q, k, v = x
        
        # (batch_size, N, num_heads * d_k)
        q_W_Q = tf.tensordot(q, self._W_Q, [[2], [0]])
        k_W_K = tf.tensordot(k, self._W_K, [[2], [0]])
        # (batch_size, N, num_heads * d_v)
        v_W_V = tf.tensordot(v, self._W_V, [[2], [0]])
        # (batch_size, num_heads, N, d_k)
        q_W_Q = split_heads(q_W_Q, self._num_heads)
        k_W_K = split_heads(k_W_K, self._num_heads)
        # (batch_size, num_heads, N, d_v)
        v_W_V = split_heads(v_W_V, self._num_heads)
        
        # (batch_size, num_heads, N, d_v)
        x = dot_product_attention(q_W_Q, k_W_K, v_W_V)
        # (batch_size, N, num_heads * d_v)
        x = combine_heads(x)

        # (batch_size, N, d_model)
        return tf.tensordot(x, self._W_O, [[2], [0]])
    
    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [62]:
d_k = 20
d_v = 24
d_model = 12
n = 5

inputs = [tf.keras.layers.Input(shape=(n, d_model,)) for _ in range(3)]
multi_head_attention = MultiHeadAttention(8, d_k, d_v)(inputs)

model = tf.keras.models.Model(
    inputs=inputs, outputs=multi_head_attention)
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(0.001),
    loss='mse')

In [63]:
Q, K, V = [np.random.randn(2, n, d_model).astype(np.float32) for _ in range(3)]

model.predict([Q, K, V]).shape

(2, 5, 12)

In [64]:
batch_size = 2
N = 10 # number of maximum context length
M = 5 # number of maximum question length
p1 = 300 # word embedding size
C = 16 # number of maximum word length
char_vocab_size = 1000
p2 = 200 # character embedding size
filter_size = 7
num_heads = 8
num_embedding_encoder_conv_layers = 4
d = 128

inputs = tf.keras.Input(shape=(N, p1 + p2))

# (batch_size, 1, N, p1 + p2)
x = tf.keras.layers.Reshape((1, N, p1 + p2))(inputs)
print(x)

for l in range(num_embedding_encoder_conv_layers):
    # (batch_size, 1, N, 128)
    print(x)
    x = tf.keras.layers.SeparableConv2D(
        d,
        kernel_size=(1, 7), padding='same')(x)

model = tf.keras.models.Model(
    inputs=inputs, outputs=x)
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(0.001),
    loss='mse')

DeferredTensor('None', shape=(?, 1, 10, 500), dtype=float32)
DeferredTensor('None', shape=(?, 1, 10, 500), dtype=float32)
DeferredTensor('None', shape=(?, 1, 10, 128), dtype=float32)
DeferredTensor('None', shape=(?, 1, 10, 128), dtype=float32)
DeferredTensor('None', shape=(?, 1, 10, 128), dtype=float32)


In [65]:
from tensor2tensor.layers.common_attention import add_timing_signal_1d

class PositionEncoding(tf.keras.layers.Lambda):
    def __init__(self, **kwargs):
        super(PositionEncoding, self).__init__(
            function=lambda x: add_timing_signal_1d(x),
            **kwargs)
    
    def compute_output_shape(self, input_shape):
        return input_shape

inputs = tf.keras.Input(shape=(N, p1 + p2))
x = PositionEncoding()(inputs)
# (batch_size, 1, N, p1 + p2)
x = tf.keras.layers.Reshape((1, N, p1 + p2))(x)

for l in range(num_embedding_encoder_conv_layers):
    # (batch_size, 1, N, d)
    x = tf.keras.layers.SeparableConv2D(
        d,
        kernel_size=(1, 7),
        padding='same')(x)
    x = tf.keras.layers.Activation('relu')(x)

# (batch_size, N, d)
x = tf.keras.layers.Reshape((N, -1))(x)
# (batch_size, N, d)
x = MultiHeadAttention(num_heads, d, d)([x] * 3)
# (batch_size, N, d)
x = tf.keras.layers.Dense(d)(x)
x = tf.keras.layers.Activation('relu')(x)
    
model = tf.keras.models.Model(
    inputs=inputs, outputs=x)
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(0.001),
    loss='mse')

In [66]:
model.predict(np.random.randn(batch_size, N, p1 + p2)).shape

(2, 10, 128)

一応出来たけど不安…少なくともdropoutは後で追加する、後layer normalization、layer dropout

NLPLearn/QANetだとmultihead attentionに畳込みを使っているのが気になる。
QANetにもTransformerにも元論文にそんな記述は見つけていない。

ただし、transformerには畳込みの実装もある。